## Day 2

Run **CarveMe** to build whole genome metabolic models of our bacteria. This will run for some while (about 90sec per species) so in the meantime you can read more about CarveMe here: https://carveme.readthedocs.io/en/latest/usage.html


In [ ]:
!carve --refseq GCF_001548235.1 -o Rothia_mucilaginosa.xml
!carve --refseq GCF_001069775.1 -o Streptococcus_cristatus.xml
!carve --refseq GCF_001069855.1 -o Streptococcus_salivarius.xml
!carve --refseq GCF_003584215.1 -o Veillonella_parvula.xml

Once the models are build, we can use parsimous Flux Balance Analysys (pFBA) from Framed https://framed.readthedocs.io/en/latest/ package to simulate growth.

In [ ]:
from framed import load_cbmodel, pFBA
from framed.experimental import medium
from framed import Environment

for f in ['Rothia_mucilaginosa','Streptococcus_cristatus','Streptococcus_salivarius','Veillonella_parvula']:
    model = load_cbmodel(f+'.xml', flavor='cobra')
    (m,solution) = medium.minimal_medium(model,min_mass_weight=True)
    Environment.from_compounds(m,exchange_format="'{}'").apply(model)
    solution = pFBA(model)
    print('organism: ',f)
    print( 'growth rate', solution.values[model.biomass_reaction] )
    print()

Now that we have the models and calculated growth rates, lets run **Smetana** to investigate the inter species metabolite exchange. You can read more about smetana here : https://smetana.readthedocs.io/en/latest/. We tell Smetana to calculate minimal media using molecular weights (--molweight), we want detailed (-d) interspecies interactions and we don't care about inorganic compounds (--exclude):

In [ ]:
!smetana --flavor cobra --molweight -d -o "smetana_out" --exclude "inorganic.txt"  "*.xml"

Lets look at the file Smetana produced.

In [ ]:
import pandas as pd
data = pd.read_csv('smetana_out_detailed.tsv',sep='\t')
print(data.head())

We want just interacions with smetana score > 0.1, which means these appeared in > 10% of the simulations.

In [ ]:
data_filtered = data[data['smetana'] > 0.1]

You can use McNet (http://mcnet.biobyte.de/) to visualize the interactions. Ask someone for a login :( and then upload the smetana file.
                                                                                                      
But anyway. Lets now run Smetana with -g option, which calculates  MIP and MRO.                                                                                                 

In [ ]:
!smetana --flavor cobra  --molweight -g -o "smetana_simple" "*.xml"

dfs = pd.read_csv('smetana_simple_global.tsv',sep='\t')
print(dfs)

MIP (7) and MRO (0.65) values suggest ... TODO

Now to continue with the analysis, lets add abiotic perturbation to the media and recalculate the growth rate and investigate change in interactions.

In [ ]:
# run the abiotic preturbations for some compounds

!smetana -a "substrates.txt" -v -n 0 --exclude "inorganic.txt" --flavor cobra --molweight -o "smetana_simple_abiotic" "*.xml"    

Once again, you can use McNet to visualize the results.

For rest of the analysis, I've randomly chosen one compound (Thymidine). Now we'll try to recalculate the growth rate for each bacteria,
by using the minimal medium for the comunity, plus the metabolites it recieves from other bacteria in community.

In [ ]:
import pandas as pd
import numpy as np

from framed import load_cbmodel, pFBA
from framed.experimental import medium
from framed import Environment
from framed import Community
from smetana import minimal_environment

df = pd.read_csv('smetana_simple_abiotic_detailed.tsv',sep='\t')
df = df[df['smetana'] > 0.1]
df_all = df

models_inCom = []
for f in ['Rothia_mucilaginosa','Streptococcus_cristatus','Streptococcus_salivarius','Veillonella_parvula']:
    models_inCom.append(load_cbmodel(f+'.xml', flavor='cobra'))

substrate_df = []

#count minimal media for community
print('Calculating minimal medium')
    
new_community = Community("", models_inCom, copy_models=False)
env = minimal_environment(new_community, verbose=True, min_mol_weight=True)
minimal_media = env.get_compounds()
    
print('Getting perturbation')
    
perturbation = ['thymd']
minimal_media.extend(perturbation)
    
to_use = [0,1,2,3]
    
names = ['Rothia_mucilaginosa','Streptococcus_cristatus','Streptococcus_salivarius','Veillonella_parvula']
#for every organism calculate growth 
for i in to_use:
    #determine received compounds from other bacteria, add it to minimal media
    m_name = names[i]
    print('Organism: %s' %m_name)
   
    rec = df_all[df_all['receiver'] == m_name]['compound'].tolist()
    #print('receiveing', rec)
    used_media = list(set(minimal_media + rec))   
        
    m_model = models_inCom[i]
    print('Applying medium')
    Environment.from_compounds(used_media).apply(m_model)
    
        
    other_reactions = set(m_model.get_exchange_reactions()) - set(env)
    new_reactions, sol = medium.minimal_medium(m_model, exchange_reactions=other_reactions, min_mass_weight=True)
    for r_id in new_reactions:
        m_model.set_lower_bound(r_id, -10)
    
    print('Running pFBA')
    solution = pFBA(m_model)
    print( 'growth rate', solution.values[m_model.biomass_reaction] )
    
    #substrate_df.append( (name[0], name[1], m_name, solution.values[m_model.biomass_reaction]) )
    #comunity, media, organism, growth
    print('______________________________________________________')
    print()
